## Setup

In [1]:
# Install spacy
"""!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install transformers
"""

'!pip install -U pip setuptools wheel\n!pip install -U spacy'

In [1]:
import spacy
import pandas as pd
from collections import Counter
from spacy import displacy
from transformers import pipeline
!python -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_sm")

2023-04-25 12:31:31.648531: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [64]:
sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.


KeyboardInterrupt: 

In [74]:
summarizer = pipeline("summarization")


No model was supplied, defaulted to t5-small and revision d769bba (https://huggingface.co/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Your max_length is set to 200, but you input_length is only 106. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)
/Users/florian/opt/anaconda3/lib/python3.9/site-packages/transformers/generation/tf_utils.py:745: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
2023-04-25 13:53:36.877722: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fe98567ff30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-04-25 13:53:36.878326: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2023-04-25 13:53:36.884088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binar

[{'summary_text': 'net outflows from 300 credit Suisse branded investment funds have reached $5.6 billion over the three weeks since the historic merger . investors had already pulled around $3 billion from the merged firm .'}]

In [81]:
pprint(summarizer(text))

Token indices sequence length is longer than the specified maximum sequence length for this model (383159 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [79]:
from pprint import pprint
pprint(text[500:1000])

('Credit Suisse funds in the week surrounding the shotgun wedding.\n'
 '\n'
 'The latest figures show an additional $2.5 billion of net outflows from '
 'Credit Suisse funds between March 23 and April 6.\n'
 '\n'
 '“It is fair to say that there is still uncertainty around the deal and the '
 'potential impact that it may have on Credit Suisse Asset Management,” Johann '
 'Scholtz, an equity analyst at Morningstar told FN.\n'
 '\n'
 'Heavy outflows from Credit Suisse funds began on March 14, the day the bank '
 'said it had discovered “materia')


In [19]:
cs_bodies = pd.read_csv('../data/bodies/bodies_Credit Suisse_Apr17.csv')
cs_bodies['body_type'] = cs_bodies['Body'].apply(lambda x: type(x))
cs_bodies = cs_bodies[cs_bodies['body_type'] == str]
cs_bodies['body_length_chr'] = cs_bodies['Body'].apply(lambda x: len(x))
body_lst = cs_bodies['Body']
text = ' '.join(cs_bodies['Body'])

In [87]:
test = 'av av dfv dfv safd sadf sd asdv asdv asdv asdv'

def analyze_sentiment(text):
    result = sentiment_analysis(text)[0]
    return result['label'], result['score']

sentiment, score = analyze_sentiment(test)

print(sentiment, score)


NEGATIVE 0.9840806126594543


In [42]:
def extract_top5_orgs(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    orgs = [ent for ent in entities if ent[1] == 'ORG']
    top5_orgs = list(pd.Series([org[0] for org in orgs]).value_counts().head(5).index)
    return top5_orgs



In [72]:
def summarizer(text):
    result = summarizer(text)[0]
    return result['label']

RecursionError: maximum recursion depth exceeded

In [60]:
print(f"5 most mentioned entities: {top5_orgs}")
print(f"Sentiment: {sentiment}, Score: {score}")

5 most mentioned entities: ['Credit Suisse', 'UBS', 'Reuters', 'SEC', "Credit Suisse's"]
Sentiment: NEGATIVE, Score: 0.9954714775085449


In [14]:
doc = nlp(cs_bodies['Body'][45])
sentence1 = list(doc.sents)[0]

In [21]:
# dependency graph
# displacy.render(sentence1, style="dep")

## Similarity

In [35]:
doc2 = nlp(cs_bodies['Body'][15])
doc2[:50]

Inflation may be weakening, but it’s not going away. That’s why pricing expert Adam Echter—who has spent almost every hour of the last three years first explaining to companies how a coming inflation would impact their business, and then helping them do something

In [36]:
doc[:50]

Simon-Kucher looks back at another spectacular year of growth, booking 21% growth in 2022 to help the global consultancy crack the half a billion euro revenue barrier.

Founded in 1985 by four high-profile German academics, Simon-Kucher today is an

In [37]:
doc.similarity(doc2)

0.832031002776474

## Entity recognition

In [ ]:
orgs = []

for ent in doc.ents:
    if ent.label_ == 'ORG':
        orgs.append(ent.text)
counted_list = Counter(orgs)

for item, count in counted_list.items():
    print(f"{item}: {count}")

# Visual representation of entities
#displacy.render(doc, style="ent")

### <font color='red'> **How to ensure entities are recongnized correctly using maybe lemmatization** </font>

## Pipelines

In [8]:
nlp = spacy.blank('en')

In [9]:
nlp.add_pipe('sentencizer')

In [15]:
doc_p = nlp(cs_bodies['Body'][45])

In [18]:
len(list(doc.sents))

19